# The Halo Mass Function

We are going to measure the Halo Mass Function from the MPAHalo table in the [millimil](http://galaxy-catalogue.dur.ac.uk:8080/Millennium/) in two ways and from two measures of halo mass. The millimil has been run in a cubic box of comovin side of $62.5 {\rm Mpc}/h$ with dark matter particles with a minnimum mass of $0.86*10^9 {\rm M}_{\odot}/h$. The assumed cosmological parameters in this simulation are: $\Omega_0 = 0.25$, $\Lambda _0 = 0.75$, $\Omega _b = 0.045$ and $h_0 = 0.73$. 


### Q.1 What is the Halo Mass Function?

We are going to read the file you previously produced and make an SQL query, so we need to import the 'eagleSqlTools' and 'numpy' in our python program. In order to check the existance of files, we also need 'os.path'.

In [ ]:
import eagleSqlTools as sql
import numpy as np
import os.path

Next a connection needs to be defined, including a username ("xyz"), password ("abc") and link to the [Database](http://virgodb.dur.ac.uk:8080/Millennium): 

In [ ]:
con = sql.connect("xyz", "abc", url="http://virgodb.dur.ac.uk:8080/Millennium")

The SQL is defined as a string (spaces do not matter here) and can be run as follows: 

In [ ]:
the_query = """select .1*(.5+floor((log10(stellarMass)+10.)/.1)) as mass,
                      log10(count(*)/power(62.5,3.)/.1) as phi
               from millimil..DeLucia2006a
               where snapnum = 63 and stellarMass > 0 
               group by .1*(.5+floor((log10(stellarMass)+10.)/.1))
               order by mass"""

data = con.execute_query(the_query)
mass = data["mass"] 
phi = data["phi"]